# Documentation of the test cases in test_aabb.cpp
### Troubleshooting
In case this notebook isn't working, make sure the matplotlib widgets are installed and enabled:
https://stackoverflow.com/a/55848505

In [108]:
%matplotlib widget

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [207]:
def get_bb(bmin, bmax):   
    phi = np.arange(1,10,2)*np.pi/4
    Phi, Theta = np.meshgrid(phi, phi)
    
    x_scale = abs(bmax[0]-bmin[0])
    y_scale = abs(bmax[1]-bmin[1])
    z_scale = abs(bmax[2]-bmin[2])

    x = (np.cos(Phi)*np.sin(Theta)+0.5) * x_scale + bmin[0]
    y = (np.sin(Phi)*np.sin(Theta)+0.5) * y_scale + bmin[1]
    z = (np.cos(Theta)/np.sqrt(2)+0.5)  * z_scale + bmin[2]
    
    return x,y,z

def calc_ray(ray_dist, inter_min, inter_max):
    ray_dir = (inter_max-inter_min)
    ray_dir = ray_dir / np.linalg.norm(ray_dir)
    ray_inv_dir = 1/ray_dir
    
    ray_org = inter_min - (ray_dir * ray_dist)     
    
    t_min = np.linalg.norm(inter_min - ray_org)
    t_max = np.linalg.norm(inter_max - ray_org)
    
    return ray_org, ray_dir, ray_inv_dir, t_min, t_max

def plot(bmin, bmax, inter_min, inter_max, ray_dist):
    bmin = np.array(bmin)
    bmax = np.array(bmax)
    inter_min = np.array(inter_min)
    inter_max = np.array(inter_max)
    
    ray_org, ray_dir, ray_inv_dir, t_min, t_max = calc_ray(ray_dist, inter_min, inter_max)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    x, y, z = get_bb(bmin, bmax)
    line = np.linspace(ray_org, ray_org + ray_dir * max_t, 100) 
    
    # plot bounding box
    ax.plot_surface(x, y, z, alpha=0.2)
    
    # plot ray origin
    ax.scatter(ray_org[0], ray_org[1], ray_org[2])
    
    # plot intersection points
    ax.scatter(inter_min[0], inter_min[1], inter_min[2])
    ax.scatter(inter_max[0], inter_max[1], inter_max[2])    
    
    # plot ray
    ax.plot(line[:,0], line[:,1], line[:,2])
    
    plt.show()
    
    return ray_org, ray_dir, ray_inv_dir, t_min, t_max

def convert_testcase(testcase, min_t, max_t):
    print(testcase, min_t, max_t)

In [209]:
min_t = 0
max_t = 3
bmin = [-1, 1, 1]
bmax = [1, 0, 2]
inter_min = [0, 1, 1.5]
inter_max = [0, 2, 1.3]
ray_dist = (max_t - min_t)/2

testcase = plot(bmin, bmax, inter_min, inter_max, ray_dist)

convert_testcase(testcase, min_t, max_t)


<ipython-input-207-e94cb0c40ebb>:18: RuntimeWarning: divide by zero encountered in true_divide
  ray_inv_dir = 1/ray_dir
<ipython-input-207-e94cb0c40ebb>:35: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 0.        , -0.47087101,  1.7941742 ]), array([ 0.        ,  0.98058068, -0.19611614]), array([        inf,  1.0198039 , -5.09901951]), 1.5000000000000002, 2.519803902718557)
